<a href="https://colab.research.google.com/github/miller00315/ia_studies/blob/main/segmentar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentação RFV

Em uma empresa, possuimos diversos tipos de clientes. Alguns retornando varias vezes na loja. Outros vindo raramente mas gastando bastante quando vem, e quaisquer outras combinações.

O objetivo desta segmentação é então permitir  que separemos os clientes pelo valor que ele traz a seu negócio e permitir comunicações direcionadas para tal.


## Dataset

Para realizar nossa analise hoje, usaremos o dataset provido em:

https://raw.githubusercontent.com/RafaelJustALittleData/Curso_Aberto/refs/heads/main/Transactions.csv

Este é um dataset 100% sintetico criado pelos criadores do curso

Primeiro o lemos usando a biblioteca pandas

In [ ]:
import pandas as pd



In [ ]:
filepath='https://raw.githubusercontent.com/RafaelJustALittleData/Curso_Aberto/refs/heads/main/Transactions.csv'

In [ ]:
Dados_Transacionais=pd.read_csv(filepath)

In [ ]:
Dados_Transacionais

,ID_CLIENT,ID_SKU,Quantity,dat_event,Valor_Unitario,Categoria,Valor_Total_Produto
0,5034,66,1,2024-01-01,75,Utensilios,75
1,7860,49,1,2024-01-01,266,Utensilios,266
2,4196,54,1,2024-01-01,211,Limpeza,211
3,9536,79,1,2024-01-01,163,Limpeza,163
4,3568,53,1,2024-01-01,187,Bebidas,187
...,...,...,...,...,...,...,...
74738,2615,7,5,2024-12-31,288,Carnes,1440
74739,2615,81,5,2024-12-31,63,Bebidas,315
74740,8157,2,5,2024-12-31,207,Carnes,1035
74741,8157,45,5,2024-12-31,151,Carnes,755


A partir destes dados transacionais devemos calcular as seguintes propriedades por consumidor:



- Recência: Quantos dias se passaram desde sua ultima compra

- Frequência: Quantas compras o individuo ja realizou

- Valor: Qual o valor medio gasto pelo individuo durante uma transação


para este calculo devemos identificar:

- Coluna que identifica a transação

- Coluna que identifica o individuo

- Coluna que identifica valor total da transação

- Coluna que identifica data da transação


**Note que para o calculo do RFV, deve se ter apenas uma linha por transação, lembre de deixar seu dataset assim**

Para nosso exemplo, consideramos o par cliente data como identificador da transação (Se a pessoa volta no mesmo dia, conta como transação unica)

In [ ]:
Identificador_Transacao='dat_event'
Identificador_Data='dat_event'
Identificador_Valor='Valor_Total_Produto'
Identificador_Cliente='ID_CLIENT'

### Preparando dataset

Vamos garantir que tenhamos apenas uma linha por transação a partir da seguinte operação

In [ ]:
Dados_Transacionais_Preparados=Dados_Transacionais.groupby([Identificador_Cliente,Identificador_Transacao],as_index=False).agg({Identificador_Data:'min',Identificador_Valor:'sum'})

In [ ]:
Dados_Transacionais_Preparados.head()

,ID_CLIENT,dat_event,Valor_Total_Produto
0,0,2024-04-27,2300
1,0,2024-06-06,2246
2,0,2024-10-24,203
3,1,2024-06-26,1626
4,1,2024-07-16,1432


Por fim, calcularemos as 3 metricas definidas.

In [ ]:
RFV = Dados_Transacionais_Preparados.groupby(Identificador_Cliente).agg(
    Frequencia       = (Identificador_Transacao, 'count'),
    Primeira_Compra  = (Identificador_Data, 'min'),
    Ultima_Compra    = (Identificador_Data, 'max'),
    Ticket_Medio     = (Identificador_Valor, 'mean')
)

In [ ]:
RFV

,Frequencia,Primeira_Compra,Ultima_Compra,Ticket_Medio
ID_CLIENT,,,,
0,3,2024-04-27,2024-10-24,1583.000000
1,3,2024-06-26,2024-12-20,1435.333333
2,2,2024-03-17,2024-05-11,1710.500000
3,4,2024-01-12,2024-08-11,1285.250000
4,2,2024-03-24,2024-12-13,1408.000000
...,...,...,...,...
9995,1,2024-12-04,2024-12-04,575.000000
9996,4,2024-01-13,2024-12-12,1972.250000
9997,3,2024-06-26,2024-08-19,835.666667


A seguir convertemos as colunas de datas para dias passados




In [ ]:
Dias_Primeira_Compra=(pd.to_datetime(RFV['Ultima_Compra']).max() - pd.to_datetime(RFV['Primeira_Compra']) ).dt.days
Dias_Ultima_Compra=(pd.to_datetime(RFV['Ultima_Compra']).max() - pd.to_datetime(RFV['Ultima_Compra']) ).dt.days

In [ ]:
RFV['Primeira_Compra']=Dias_Primeira_Compra

In [ ]:
RFV['Ultima_Compra']=Dias_Ultima_Compra

In [ ]:
RFV

,Frequencia,Primeira_Compra,Ultima_Compra,Ticket_Medio
ID_CLIENT,,,,
0,3,248,68,1583.000000
1,3,188,11,1435.333333
2,2,289,234,1710.500000
3,4,354,142,1285.250000
4,2,282,18,1408.000000
...,...,...,...,...
9995,1,27,27,575.000000
9996,4,353,19,1972.250000
9997,3,188,134,835.666667


E com isto temos o modelo RFV construido, ja tendo seus dados extraidos.

Como interpretar este dataset?

- Frequencia: Quantas transações distintas o consumidor fez

- Primeira_Compra: Quantos dias faz desde que este fez a primeira compra

- Ultima compra: A quantos dias este esta sem comprar

- Ticket_Medio: Valor medio gasto em reais

Então por exemplo o cliente de identificação  9999  voltou 2 vezes na loja, esta a 29 dias sem comprar e gastou na media 1503 reais cada vez que veio

## Estrategias de segmentação


Para segmentar estes podemos adotar duas possiveis estrategias.

Segmentação via kmeans

Segmentação via quantis



### Segmentação via quantis

Normalmente na empresa que voce esta atuando, ja é bem definido um conjunto de grupos que deseja se trabalhar.

Por exemplo , atua com o conceito de descompromissados potenciais, valioso e vips

Neste cenario, ja temos o numero de clusters bem definidos, e portanto ambos os metodos são igualmente viaveis

Vamos assumir a seguinte estrutura.

4 grupos

Descompromissados, Potenciais, Valiosos,Vips

In [ ]:
RFV=RFV.rename(columns={'Ultima_Compra':'Recencia','Ticket_Medio':'Valor'})

In [ ]:
Clusters=4

In [ ]:
Grupos=[None,'Descompromissados','Potenciais','Valiosos','Vip'] # Criamos o primeiro campo como none pois o python começa com o indice 0

Para calcular os quantis, usamos o metodo quantile

In [ ]:
Indice_Quantil=[]
Quantil_Recencia=[]
Quantil_Frequencia=[]
Quantil_Valor=[]
for i in range(Clusters):
  Indice_Quantil.append(i+1)
  Quantil_Frequencia.append(RFV['Frequencia'].quantile(i/Clusters))
  Quantil_Recencia.append(RFV['Recencia'].quantile(i/Clusters))
  Quantil_Valor.append(RFV['Valor'].quantile(i/Clusters))


In [ ]:
Cortes=pd.DataFrame({'Score':Indice_Quantil,'R_Quantil':Quantil_Recencia,'F_Quantil':Quantil_Frequencia,'V_Quantil':Quantil_Valor})

In [ ]:
Cortes

,Score,R_Quantil,F_Quantil,V_Quantil
0,1,0.0,1.0,58.000000
1,2,32.0,2.0,1002.100000
2,3,79.0,3.0,1326.708333
3,4,153.0,4.0,1678.250000


Em seguida, juntamos os scores de forma que a medida do cliente deva ser maior .

Para a recencia especificamente, nos temos que inverter (Menor recencia deve ser maior score)

In [ ]:
Corte_Recencia=Cortes[['Score','R_Quantil']].rename(columns={'Score':'R_Quantile','R_Quantil':'Recencia'} )
Corte_Recencia['Recencia']=Corte_Recencia['Recencia'].astype('int')

Corte_Recencia['R_Quantile']=Corte_Recencia['R_Quantile'].max() -Corte_Recencia['R_Quantile'] +1
RFV=pd.merge_asof(RFV.sort_values('Recencia'),Corte_Recencia  )

frequencia

In [ ]:
Corte_Frequencia=Cortes[['Score','F_Quantil']].rename(columns={'Score':'F_Quantile','F_Quantil':'Frequencia'} )
Corte_Frequencia['Frequencia']=Corte_Frequencia['Frequencia'].astype('int')
RFV=pd.merge_asof(RFV.sort_values('Frequencia'),Corte_Frequencia  )

Valor

In [ ]:
Corte_Valor=Cortes[['Score','V_Quantil']].rename(columns={'Score':'V_Quantile','V_Quantil':'Valor'} )
RFV=pd.merge_asof(RFV.sort_values('Valor'),Corte_Valor  )

In [ ]:
RFV

,Frequencia,Primeira_Compra,Recencia,Valor,R_Quantile,F_Quantile,V_Quantile
0,1,343,343,58.0,1,1,1
1,1,327,327,59.0,1,1,1
2,1,314,314,59.0,1,1,1
3,1,266,266,61.0,1,1,1
4,1,165,165,61.0,1,1,1
...,...,...,...,...,...,...,...
9513,1,126,126,3974.0,2,1,4
9514,1,265,265,4032.0,1,1,4
9515,1,277,277,4036.0,1,1,4
9516,1,277,277,4396.0,1,1,4


Por fim geramos os scores finais.

In [ ]:
Pesos={'Recencia':1,'Frequencia':1,'Valor':1}
RFV['Score_Final']=(RFV['R_Quantile'] * Pesos['Recencia'] + RFV['F_Quantile'] * Pesos['Frequencia'] + RFV['V_Quantile'] * Pesos['Valor']  )/ (Pesos['Recencia']+Pesos['Frequencia'] + Pesos['Valor'])

In [ ]:
RFV['Segmento'] = [Grupos[i] for i in RFV['Score_Final'].astype('int')]

In [ ]:
RFV.sort_values('Score_Final')

,Frequencia,Primeira_Compra,Recencia,Valor,R_Quantile,F_Quantile,V_Quantile,Score_Final,Segmento
32,1,344,344,138.000000,1,1,1,1.0,Descompromissados
30,1,265,265,134.000000,1,1,1,1.0,Descompromissados
13,1,294,294,80.000000,1,1,1,1.0,Descompromissados
12,1,302,302,76.000000,1,1,1,1.0,Descompromissados
11,1,326,326,76.000000,1,1,1,1.0,Descompromissados
...,...,...,...,...,...,...,...,...,...
8430,5,254,16,1993.200000,4,4,4,4.0,Vip
8441,4,275,2,1996.250000,4,4,4,4.0,Vip
8470,4,264,15,2009.000000,4,4,4,4.0,Vip
8473,4,313,26,2009.000000,4,4,4,4.0,Vip


e assim geramos a segmentação RFV, onde temos um individuo Vip que  realizou 9 compras, sua ultima a 12 dias, gastando quase 2000 reais por compra na media, enquanto ha individuos descompromissados que so fizeram uma transação no valor de 138 reais.


Estas informações permitem que voce faça campanhas dedicadas adaptando a forma de se comunicar baseado no tipo de cliente que voce vai conversar

### Bonus: Segmentação kmeans

Referencia do metodo: https://www.geeksforgeeks.org/k-means-clustering-introduction/

In [ ]:
RFV_K_Means=RFV[['Recencia','Frequencia','Valor']]

O modelo kmeans tem como hipotese variancia constante entre variaveis, por causa disso a normalização adotada

In [ ]:
RFV_K_Means_Normalized=(RFV_K_Means-RFV_K_Means.mean() )/(RFV_K_Means.std() )

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(RFV_K_Means_Normalized)


In [ ]:
kmeans.labels_

array([1, 1, 1, ..., 2, 2, 2], dtype=int32)

In [ ]:
RFV_K_Means['Cluster']=kmeans.labels_

<ipython-input-32-1912a1c09f0e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RFV_K_Means['Cluster']=kmeans.labels_


Comparando ambos

In [ ]:
RFV.groupby('Segmento')[['Recencia','Frequencia','Valor']].mean()

,Recencia,Frequencia,Valor
Segmento,,,
Descompromissados,205.584768,1.538411,820.514790
Potenciais,121.989200,2.689985,1359.541786
Valiosos,46.905584,4.172646,1545.702291
Vip,15.202643,4.960352,1911.842462


In [ ]:
RFV_K_Means.groupby('Cluster')[['Recencia','Frequencia','Valor']].mean()

,Recencia,Frequencia,Valor
Cluster,,,
0,54.546233,5.106391,1372.112757
1,245.187079,1.847191,1191.778642
2,96.816692,2.371224,2054.650691
3,69.184651,2.565013,993.388265




Caso voce queira dar mais peso a alguma das variaveis, voce pode modificar os pesos da media ponderada, ou ate multiplicar as variaveis posnormalização para aumentar a influencia  de algum destes atributos. e a partir disto, segmentar seus clientes baseado no que é importante para a empresa.